In [1]:
import pandas as pd
import numpy as np
import psycopg2
from psycopg2 import sql 
#!ln -s /var/run/postgresql/.s.PGSQL.5432 /tmp/.s.PGSQL.5432'

In [2]:
df = pd.read_csv('../Datamyne_Delivery_7_28_2017/Data/meshintel-datamyne_d_b_family_tree.csv')

/home/ram22/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2683: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
df.head()

,duns_number,business_name,headquarter_duns_number,headquarter_parent_business_name,domestic_ultimate_duns_number,domestic_ultimate_business_name,global_ultimate_duns_number,global_ultimate_business_name
0,1000306,"Shrut & Asch Leather Company, Inc.",NaN,NaN,NaN,NaN,NaN,NaN
1,1000355,"Harrison Specialty Co., Inc.",1000355.0,"Harrison Specialty Co., Inc.",001000355,"Harrison Specialty Co., Inc.",1000355.0,"Harrison Specialty Co., Inc."
2,1000363,"Louis M. Gerson Co., Inc.",1000363.0,"Louis M. Gerson Co., Inc.",001000363,"Louis M. Gerson Co., Inc.",1000363.0,"Louis M. Gerson Co., Inc."
3,1001478,National Nonwovens Inc.,1001478.0,National Nonwovens Inc.,001001478,National Nonwovens Inc.,1001478.0,National Nonwovens Inc.
4,1002047,"Hytex Industries, Inc.",NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
# Global ultimates which identify what we want
global_ultimates = [217304393, # BAE SYSTEMS PLC 
                    778557603, # CENTRICA PLC
                    790428754, # Chobani, LLC
                    305422255, # DONG Energy Power A/S
                    1315704, # E. I. Du Pont De Nemours and Company
                    12430880, # Herny Schein, Inc.
                    544957314, # Huawei Investment and Holding Co., Ltd.
                    671584550, # INDORAMA VENTURES PUBLIC COMPANY LIMITED
                    81466849, # Microsoft Corportation
                    690546460, # MITSUBISHI CORPORATION
                    480708874, # Nestle S.A.
                    6988414, # Newmont Mining Corporation
                    50957364, # Nike, Inc.
                    485609796, # Novartis AG
                    489840363, # Qiagen N.V.
                    144356508, # Qualcomm Incorporated
                    480008226, # Roche Holding AG
                    3296175, # The Coca-Cola Company
                    210300901, # UNILEVER PLC
                    289936783, # VODAFONE GROUP PLC
                    51957769, # Wal-Mart Stores, Inc.
                    387451045, # Louis Dreyfus Holding B.V.
                    79942718, # Alphabet, Inc.
                    60902413, # Google Inc.
                    461026429, # IBERDROLA, SOCIEDAD ANONIMA
                    238980408] #GLAXOSMITHKLINE PLC
domestic_ultimates = [366470602] # CONSOLIDATED CONTRACTORS INTERNATIONAL COMPANY S A L; 315275255 PUMA SE at this level only points at itself
headquarters = [315275255, 402491757, 794661801] # Puma SE; Heineken N.V.; Nespresso Usa, Inc.; 654292358 for Huawei Techonologies Co. Ltd.
entities = [862932619] # Bajaj Auto Corporation
# These make 31
print(len(global_ultimates) + len(domestic_ultimates) + len(headquarters) + len(entities))
# Not looked for: Klockner & Co, F. Hoffmann-La Roche Limited; Royal Philips; Reuters Market Light, India; Nykredit AS; Northwell Health; Dogan Broadcasting
# Not found: Mumm, Dignity Health , VTB Bank
# Problem with ROCHE
# Those are 11 (not double counting ROCHE). Plus 31 = 42. But I added Alphabet in global_ultimates.

31


In [5]:
# Creates a list of the companies we want if they exist as entities in D&B data
targetDf = df[df['duns_number'].isin(global_ultimates)].copy()
targetDf['foundIn'] = 'global ultimate'

temp = df[df['duns_number'].isin(domestic_ultimates)].copy()
temp['foundIn'] = 'domestic ultimate'
targetDf  = targetDf.append(temp, ignore_index=True)

temp = df[df['duns_number'].isin(headquarters)].copy()
temp['foundIn'] = 'headquarter'
targetDf  = targetDf.append(temp, ignore_index=True)

temp = df[df['duns_number'].isin(entities)].copy()
temp['foundIn'] = 'entity'
targetDf  = targetDf.append(temp, ignore_index=True)


In [6]:
targetDf['foundIn'].value_counts(dropna=False)

global ultimate      23
domestic ultimate     1
headquarter           1
entity                1
Name: foundIn, dtype: int64

In [7]:
names = targetDf['business_name'].value_counts(dropna=False)
names.sort_index()

BAE SYSTEMS PLC                                         1
Bajaj Auto Corporation                                  1
CENTRICA PLC                                            1
CONSOLIDATED CONTRACTORS INTERNATIONAL COMPANY S A L    1
Chobani, LLC                                            1
DONG Energy Power A/S                                   1
E. I. Du Pont De Nemours and Company                    1
GLAXOSMITHKLINE PLC                                     1
Google Inc.                                             1
Henry Schein, Inc.                                      1
Huawei Investment And Holding Co., Ltd.                 1
INDORAMA VENTURES PUBLIC COMPANY LIMITED                1
MITSUBISHI CORPORATION                                  1
Microsoft Corporation                                   1
Nestlé S.A.                                             1
Newmont Mining Corporation                              1
Nike, Inc.                                              1
Novartis AG   

In [10]:
print_full(df.columns)

['duns_number', 'business_name', 'headquarter_duns_number', 'headquarter_parent_business_name', 'domestic_ultimate_duns_number', 'domestic_ultimate_business_name', 'global_ultimate_duns_number', 'global_ultimate_business_name']


In [11]:
dbProfiles = pd.read_csv('/home/ram22/Dropbox/dataProjects/valueChains/meshintel/ozniras/datamyneDelivery20170728/02_dataCreated/meshintel-datamyne_d_b_all_companies_all.csv')

In [12]:
print_full(dbProfiles.columns)

['duns_number', 'business_name', 'tradestyle_name', 'registered_address_indicator', 'street_address', 'street_address_2', 'city_name', 'state_province_name', 'country_name', 'db_city_code', 'db_us_county_code', 'db_state_code', 'state_province_abbreviation', 'db_country_code', 'postal_code', 'db_continent_code', 'mailing_address', 'mailing_city_name', 'mailing_county_name', 'mailing_state_province_name', 'mailing_country_name', 'db_mailing_city_code', 'db_mailing_county_code', 'db_mailing_state_province_code', 'mailing_state_province_abbreviation', 'db_mailing_country_code', 'mailing_postal_code', 'db_mailing_continent_code', 'national_identification_number', 'national_identification_type_code', 'country_access_code', 'telephone_number', 'cable_telex_number', 'facsimile_number', 'chief_executive_officer_name', 'chief_executive_officer_title', 'line_of_business', 'us_1987_sic_1', 'us_1987_sic_2', 'us_1987_sic_3', 'us_1987_sic_4', 'us_1987_sic_5', 'us_1987_sic_6', 'primary_local_activity

In [13]:
# Pulls the D&B profiles for the companies we want if they exist as entities in D&B data
targetdbProfiles = dbProfiles[dbProfiles['duns_number'].isin(global_ultimates)].copy()
targetdbProfiles['foundIn'] = 'global ultimate'

temp = dbProfiles[dbProfiles['duns_number'].isin(domestic_ultimates)].copy()
temp['foundIn'] = 'domestic ultimate'
targetdbProfiles  = targetdbProfiles.append(temp, ignore_index=True)

temp = dbProfiles[dbProfiles['duns_number'].isin(headquarters)].copy()
temp['foundIn'] = 'headquarter'
targetdbProfiles  = targetdbProfiles.append(temp, ignore_index=True)

temp = dbProfiles[dbProfiles['duns_number'].isin(entities)].copy()
temp['foundIn'] = 'entity'
targetdbProfiles  = targetdbProfiles.append(temp, ignore_index=True)


In [14]:
targetdbProfiles[['business_name', 'street_address', 'country_name', 'us_1987_sic_1', 'year_started', 'sales_volume_reliability_code', 'sales_volume_us_dollars', 'employees_here', 'employees_here_reliability_code', 'employees_total', 'employees_total_reliability_code']]

,business_name,street_address,country_name,us_1987_sic_1,year_started,sales_volume_reliability_code,sales_volume_us_dollars,employees_here,employees_here_reliability_code,employees_total,employees_total_reliability_code
0,Nestlé S.A.,Avenue Nestlé 55,SWITZERLAND,6722.0,1867.0,0.0,9.219637e+10,0.0,2.0,328000.0,2.0
1,CENTRICA PLC,Millstream,ENGLAND,4932.0,1997.0,0.0,3.613691e+10,0.0,2.0,34970.0,0.0
2,The Coca-Cola Company,1 Coca Cola Plz NW,USA,2087.0,1886.0,0.0,4.429400e+10,8000.0,0.0,123200.0,0.0
3,Google Inc.,1600 Amphitheatre Pkwy,USA,4813.0,1998.0,2.0,6.749010e+10,250.0,0.0,61814.0,0.0
4,E. I. Du Pont De Nemours and Company,974 Centre Rd,USA,2879.0,1802.0,0.0,2.513000e+10,6000.0,0.0,52000.0,0.0
5,Qiagen N.V.,Spoorstraat 50,NETHERLANDS,6733.0,2007.0,0.0,1.344777e+09,40.0,2.0,4339.0,0.0
6,GLAXOSMITHKLINE PLC,G S K House,ENGLAND,2834.0,1830.0,0.0,4.354807e+10,0.0,2.0,97401.0,0.0
7,"Wal-Mart Stores, Inc.",702 SW 8th St,USA,5311.0,1945.0,0.0,4.821300e+11,11700.0,0.0,2300000.0,0.0
8,MITSUBISHI CORPORATION,"2-3-1, MARUNOUCHI",JAPAN,5149.0,1950.0,0.0,6.352385e+10,0.0,2.0,58470.0,0.0
9,BAE SYSTEMS PLC,6 Carlton Gardens,ENGLAND,6719.0,1977.0,0.0,3.399993e+10,0.0,2.0,92000.0,0.0


In [15]:
targetdbProfiles.to_csv('../02_dataCreated/targetDBprofiles.csv', index=False)

In [16]:
# Pulls all the children for the companies we want 
targetChildren = df[df['global_ultimate_duns_number'].isin(global_ultimates)].copy()
targetChildren['foundIn'] = 'global ultimate'

temp = df[df['domestic_ultimate_duns_number'].isin(domestic_ultimates)].copy()
temp['foundIn'] = 'domestic ultimate'
targetChildren  = targetChildren.append(temp, ignore_index=True)

temp = df[df['headquarter_duns_number'].isin(headquarters)].copy()
temp['foundIn'] = 'headquarter'
targetChildren  = targetChildren.append(temp, ignore_index=True)

temp = df[df['duns_number'].isin(entities)].copy()
temp['foundIn'] = 'entity'
targetChildren  = targetChildren.append(temp, ignore_index=True)


In [17]:
targetChildren.to_csv('../02_dataCreated/targetChildren.csv', index=False)

In [18]:
childrendbProfiles = dbProfiles[dbProfiles['duns_number'].isin(targetChildren['duns_number'])].copy()
childrendbProfiles = childrendbProfiles[childrendbProfiles['country_name'] == 'USA']

In [19]:
childrendbProfiles.to_csv('../02_dataCreated/childrendbProfiles.csv', index=False)

In [20]:
df[df['headquarter_duns_number'] == 402491757]

,duns_number,business_name,headquarter_duns_number,headquarter_parent_business_name,domestic_ultimate_duns_number,domestic_ultimate_business_name,global_ultimate_duns_number,global_ultimate_business_name
182212,211242996,SUNRISE ACQUISITIONS LTD,402491757.0,Heineken N.V.,211242996,SUNRISE ACQUISITIONS LTD,486538200.0,L'Arche Holding SA
220759,300676959,BRAU UNION AG,402491757.0,Heineken N.V.,300676959,BRAU UNION AG,409856346.0,L'Arche Green N.V.
266942,382043461,HEINEKEN FRANCE,402491757.0,Heineken N.V.,3.82043e+08,HEINEKEN FRANCE,409856346.0,L'Arche Green N.V.
273866,406637538,Heineken Brouwerijen B.V.,402491757.0,Heineken N.V.,4.09856e+08,L'Arche Green N.V.,409856346.0,L'Arche Green N.V.
703230,813177862,"Grupo Cuauhtémoc Moctezuma, S.A. de C.V.",402491757.0,Heineken N.V.,813177862,"Grupo Cuauhtémoc Moctezuma, S.A. de C.V.",409856346.0,L'Arche Green N.V.
758197,872007695,Brasserie Nationale D' Haiti S.A.,402491757.0,Heineken N.V.,872007695,Brasserie Nationale D' Haiti S.A.,409856346.0,L'Arche Green N.V.
758827,873200182,"Desnoes & Geddes, Limited",402491757.0,Heineken N.V.,873200182,"Desnoes & Geddes, Limited",409856346.0,L'Arche Green N.V.


In [22]:
# These are companies pulled in BoL that were matched to D&B
dbBolProfiles = pd.read_csv('/home/ram22/Dropbox/dataProjects/valueChains/meshintel/ozniras/datamyneDelivery20170728/02_dataCreated/meshintel-datamyne_bol_d_b_profiles_all.csv')

In [23]:
inBoLnotinDB = pd.Series(list(set(dbBolProfiles.consignee_d_b_duns_number.astype(int)) - set(dbProfiles.duns_number.astype(int)))) 
inBoLnotinDB.to_csv('../02_dataCreated/inBoLnotinDB.csv', index=False)
len(inBoLnotinDB)

1768

In [24]:
conn = psycopg2.connect(database = "datamyne20170728", user = "ram22")
cur = conn.cursor()

def createSql(incoming = '', out = ''):

    infile = open(incoming)
    varNames = infile.readline()
    infile.close()
    
    # get rid of \n at end of line and text delimeter, then create list
    varNames = varNames[:-2].replace('"', '').split(',')
    varNames = ['consignee_d_b_' + i for i in varNames]

    cur.execute(sql.SQL("DROP TABLE IF EXISTS {0};").format(sql.Identifier(out)))

    i = 0 
    for field in varNames:
        if i == 0:
            cur.execute(sql.SQL("CREATE TABLE {0} ({1} varchar);").format(sql.Identifier(out), sql.Identifier(field)))
            i += 1
        else:
            cur.execute(sql.SQL("ALTER TABLE {0} ADD {1} varchar;").format(sql.Identifier(out), sql.Identifier(field)))

nameFile = '../02_dataCreated/childrendbProfiles.csv'
outSQL = 'children_db_profiles'
createSql(nameFile, outSQL)
inputFile = open(nameFile, 'r')
cur.copy_expert("COPY children_db_profiles FROM STDIN WITH CSV HEADER", inputFile)
inputFile.close()

nameFile = '../02_dataCreated/targetChildren.csv'
outSQL = 'target_children'
createSql(nameFile, outSQL)
inputFile = open(nameFile, 'r') 
cur.copy_expert("COPY target_children FROM STDIN WITH CSV HEADER", inputFile)
inputFile.close()

conn.commit()
conn.close()


In [9]:
def print_full(x):
    pd.set_option('display.max_rows', len(x))
    print(list(x))
    pd.reset_option('display.max_rows')